## Riddler Classic: June 28

https://fivethirtyeight.com/features/whats-your-best-scrabble-string/

### Prompt

_The game of Scrabble has 100 tiles — 98 of these tiles contain a letter and a score, and two of them are wildcards worth zero points. At home on a lazy summer day with a bag of these tiles, you decide to play the Superstring Scrabble Challenge. Using only the 100 tiles, you lay them out into one long 100-letter string of your choosing. You look through the string. For each word you find, you earn points equal to its score. Once you find a word, you don’t get any points for finding it again. The same tile may be used in multiple, overlapping words. So ‘“theater” includes “the,” “heat,” “heater,” “eat,” “eater,” “ate,” etc._

_The super challenge: What order of tiles gives you the biggest score? (The blank tiles are locked into the letter they represent once you’ve picked it.)_

_The winner, and inaugural Wordsmith Extraordinaire of Riddler Nation, will be the solver whose string generates the most points. You should use [this word list](https://norvig.com/ngrams/enable1.txt) to determine whether a word is valid._

_For reference, this is the distribution of letter tiles in the bag, by their point value:_

- _0: ?×2_
- _1: E×12 A×9 I×9 O×8 N×6 R×6 T×6 L×4 S×4 U×4_
- _2: D×4 G×3_
- _3: B×2 C×2 M×2 P×2_
- _4: F×2 H×2 V×2 W×2 Y×2_
- _5: K_
- _8: J X_
- _10: Q Z_

Scrabble dictionary txt: https://norvig.com/ngrams/enable1.txt

### Strategy

100 factorial is roughly 9.33e157, so brute force string checking is probably out of the question.  Instead. we'll have to randomly generate tile strings from the available tilesets and check them as we go.

0. Download Scrabble wordlist and make tileset
1. Generate random tile string
2. Split tile string into ngram combinations and filter ngram set by words in the Scrabble dictionary
    - Longest possible Scrabble board word is 15 letters
    - Longest word in the Scrabble word list is 28 letters
3. Make scoring functions, score remaining set of valid words and get total score
    - Use sets to make sure the same word is not scored twice + for efficient wordlist lookup
    - Have two scoring mechanisms to handle if wildcard is present/absent in ngram
    - Wildcards are locked in once they are used as a specific letter
4. Repeat 1-3 for random tile strings

In [1]:
import numpy as np
import pandas as pd
import itertools
import random
import re

### 0. Download Scrabble wordlist and make tileset

In [2]:
# Load scrabble wordlist
WORDLIST_FILE = 'words.txt'
with open(WORDLIST_FILE) as f:
    WORDLIST = set(f.read().split('\n'))
    
sorted(list(WORDLIST))[:5]

['aa', 'aah', 'aahed', 'aahing', 'aahs']

In [3]:
# Make tile string
# @ and # represent two different wildcards
TILES = ('e'*12) + ('ai'*9) + ('o'*8) + ('nrt'*6) + ('lsud'*4) + ('g'*3) + ('bcmpfhvwy'*2) + 'kjxqz@#'

print(len(TILES))
TILES

100


'eeeeeeeeeeeeaiaiaiaiaiaiaiaiaioooooooonrtnrtnrtnrtnrtnrtlsudlsudlsudlsudgggbcmpfhvwybcmpfhvwykjxqz@#'

In [5]:
# What is the longest word in the wordlist?
max(list(map(len, WORDLIST)))

28

### 1. Make random tilestring generator

In [6]:
def get_tilestring(tiles):
    '''Samples without replacement from tiles, then joins output list back into a string and returns result.'''
    return ''.join(random.sample(tiles, k=len(tiles)))

get_tilestring(TILES)

'cbmpreztad#tkhiaqrsarougrtelaeojgeiaehaniolbunanwiuyviifmenstretsepeeodas@oloneoddwnvaguexofylitrici'

### 2. Split random tilestring into n-gram set and filter by Scrabble dictionary

In [7]:
# Function to find all possible n-grams of each length
# Adapted from http://www.locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
def get_ngrams(input_string, n, min_length=2, wordlist=WORDLIST):
    '''
    Returns a set of all valid ngrams up to max length n from an input word.
    Leaves in ngrams with wildcards. 
    '''
    # Create and join ngrams in a set
    result = set()
    for i in range(min_length, n):
        tuple_list = list(zip(*[input_string[x:] for x in range(i)]))
        ngram_list = [''.join(x) for x in tuple_list]
        result.update(ngram_list)
    
    # Filter out nonvalid words, but leave in wildcard ngrams
    wildcard_ngrams = {x for x in result if '#' in x or '@' in x}
    non_wildcard_ngrams = wordlist.intersection(result)
    
    return wildcard_ngrams.union(non_wildcard_ngrams)

get_ngrams('#heat@r', 15)

{'#h',
 '#he',
 '#hea',
 '#heat',
 '#heat@',
 '#heat@r',
 '@r',
 'at',
 'at@',
 'at@r',
 'eat',
 'eat@',
 'eat@r',
 'he',
 'heat',
 'heat@',
 'heat@r',
 't@',
 't@r'}

### 3. Make scoring functions, score remaining set of valid words and get total score

In [8]:
# Make scoring dict for each letter
letter_scores = {"a": 1 , "b": 3 , "c": 3 , "d": 2 , "e": 1 , 
                 "f": 4 , "g": 2 , "h": 4 , "i": 1 , "j": 8 , 
                 "k": 5 , "l": 1 , "m": 3 , "n": 1 , "o": 1 , 
                 "p": 3 , "q": 10, "r": 1 , "s": 1 , "t": 1 ,
                 "u": 1 , "v": 4 , "w": 4 , "x": 8 , "y": 4 , 
                 "z": 10}

In [9]:
# Make simple scoring functions for each case
def score_word(input_word, scoring_dict=letter_scores):
    '''Applies scoring_dict for each letter in the word and returns the total sum.'''
    return sum([scoring_dict[x] for x in input_word])


def score_set(input_set):
    '''Scores all words in the sets and returns the total sum'''
    return sum(list(map(score_word, input_set)))


def score_wildcard_set(input_set, wordlist=WORDLIST):
    
    sub_options = list(itertools.product([chr(97 + x) for x in range(26)], repeat=2))
    sub_scores = []
    
    for i, j in sub_options:
        substituted = {x.replace('@', i).replace('#', j) for x in input_set}.intersection(wordlist)
        sub_scores.append(score_set(substituted))
        
    best_score, best_sub = sorted(zip(sub_scores, sub_options))[-1]
    # print(f"Best substitution for '@' and '#' were {best_sub} respectively")
    
    return best_score

# Make wrapper scoring function to handle all cases
def get_score(input_set, wordlist=WORDLIST):
    '''
    Applies score_word to every element of input_set and returns the total score.
    '''
    # Score non-wildcard words
    non_wild_words = {x for x in input_set if '@' not in x and '#' not in x}
    non_wild_sum = score_set(non_wild_words)
    
    # Score wildcard words
    wild_words = {x for x in input_set if '@' in x or '#' in x}
    wild_sum = score_wildcard_set(wild_words)
    
    return non_wild_sum + wild_sum

get_score(get_ngrams('#heat@r', 15))

65

In [10]:
# Verify that sum is correct 
test_set = get_ngrams('#heat@r', 15)
{x.replace('@', 'h').replace('#', 's') for x in test_set}.intersection(WORDLIST)

{'at', 'eat', 'eath', 'he', 'heat', 'heath', 'sh', 'she', 'shea', 'sheath'}

### 4. Repeat 1-3 on multiple random tilestrings

In [11]:
best_score = 0
best_tileset = ''

for i in range(100):
    tiles = get_tilestring(TILES)
    ngrams = get_ngrams(tiles, 15)
    score = get_score(ngrams)
    
    print(f"READING: {tiles}", end='\r')
    
    if score > best_score:
        best_score = score
        best_tileset = tiles
        
print("Best tiles:", best_tileset)
print("Best score:", best_score)

Best tiles: qvarwtodsgid#edoa@jnilhaofuirarianhowobisnaifdncvbeeenixyutrorkaeillsetamreleucuszegonmeyeagtopitpte
Best score: 313
